In [3]:
from rdflib import Graph, Literal, BNode, Namespace, RDF, URIRef
from rdflib.namespace import DC, FOAF
import spotlight
import urllib, json
import pandas as pd
from collections import Counter

In [4]:
dfdbpedia = pd.read_csv("dbpedia3.csv")
dfdbpedia['1'][0]

'Acute cholecystitis'

In [147]:
len(dfdbpedia)

218

In [134]:
cn = Namespace("http://ai.symptomchecking.org/condtion/")

In [146]:
sdg = Graph()
for idx in dfdbpedia.index:
    if(220<=idx<223):
        resourcename = dfdbpedia['1'][idx].replace(':','').replace(' ','_')
        resourceobj = URIRef("http://ai.symptomchecking.org/condtion/"+resourcename)
        sameas = URIRef('http://www.w3.org/2002/07/owl#sameAs')
        
        arname = URIRef(dfdbpedia.loc[idx, "dbp"].split(',')[0][1:])
        
        linksTo = URIRef(cn.linkTo)
        signOf = URIRef(cn.signOf)
        anatomicOn = URIRef(cn.anatomicOn)
        drug = URIRef(cn.drugTo)
        symptomOf = URIRef(cn.symptomOf)
        
        disease = URIRef('http://dbpedia.org/ontology/Disease')
        wikidisease = URIRef('https://www.wikidata.org/wiki/Q12136')
        place = URIRef('http://dbpedia.org/ontology/PopulatedPlace')
        agent = URIRef('http://dbpedia.org/ontology/Person')
        book = URIRef('http://dbpedia.org/ontology/Book')
        anat = URIRef('http://dbpedia.org/ontology/AnatomicalStructure')
        wikianat = URIRef('https://www.wikidata.org/wiki/Q4936952')
        thing = URIRef('http://dbpedia.org/ontology/Thing')
        symp = URIRef('http://dbpedia.org/resource/Category:Symptoms_and_signs')
        presign = URIRef('http://purl.org/dc/terms/subject')
        abstract = URIRef("http://purl.org/dc/terms/abstract")
        drug = URIRef('http://dbpedia.org/ontology/Drug')
        condition = URIRef('http://umbel.org/umbel/rc/AilmentCondition')
        
        diseaseProperty = URIRef('http://dbpedia.org/property/diseasesdb')
        
        sdg.add((resourceobj, RDF.type, disease))
        sdg.add((resourceobj, sameas, arname))
        # finding abstract description
        dbpdialink = arname.replace('resource','data')+'.json'
        print(dbpdialink)
        icresponse = urllib.urlopen(dbpdialink)
        jicdata = json.loads(icresponse.read())
        firstkey=jicdata[dfdbpedia.loc[idx, "dbp"].split(',')[0][1:]]
        if 'http://dbpedia.org/ontology/abstract' in firstkey:
            secondkey = firstkey['http://dbpedia.org/ontology/abstract']
            for abstract in secondkey:
                if abstract['lang'] == 'en':
                    dbpediaText=abstract['value']
                    sdg.add((resourceobj, DC.abstract, Literal(dbpediaText, lang="en")))
        newlistterms = list(set(dfdbpedia.loc[idx, "dbp"].split(',')[1:]))
        for term in newlistterms:
            newuri = URIRef(term.split(' ')[1])
            g = Graph()
            g.parse(newuri)
            disflag = 0
            for disObj in g.objects(subject=newuri, predicate=diseaseProperty):
                if disObj is not None:
                    disflag = 1
                    sdg.add( (newuri, symptomOf, resourceobj, ) )
                    sdg.add( (newuri, RDF.type, symp) )
                break
            if disflag == 0 and newuri in g.subjects(object= anat, predicate=RDF.type):
                sdg.add( (resourceobj, anatomicOn, newuri) )
                sdg.add( (newuri, RDF.type, anat) )
            else:
                if disflag == 0:
                    for obj in g.objects (subject=newuri, predicate=RDF.type):
                        if drug in obj:
                            print obj
                            sdg.add( (newuri, drug, resourceobj) )
                            sdg.add( (newuri, RDF.type, obj) )
                        else:
                            if disease in obj:
                                print obj
                                sdg.add( (resourceobj, linksTo, newuri) )
                                sdg.add( (newuri, RDF.type, disease) )
                            else:
                                if condition in obj:
                                    sdg.add( (newuri, symptomOf, resourceobj) )
                                    sdg.add( (newuri, RDF.type, symp) )
                                else:
                                    sdg.add( (newuri, signOf, resourceobj) )
                                    sdg.add( (newuri, RDF.type, obj) )
        # ntriples, n3, turtle   
        # print sdg.serialize(format='turtle')
        filepath=resourcename+'.ttl'
        sdg.serialize(destination=filepath, format='turtle')


graph = Graph()

graph.parse('acne.ttl', format="turtle")
graph.parse('Acute_cholecystitis.ttl', format="turtle")


In [148]:
import os
import glob

graph = Graph()
def mergttlfile(gh):
    fileList=glob.glob("*.ttl")
    dfList=[]
    colnames=["condition","symptom"]
    print(len(fileList))
    for filename in fileList:
        #print(filename)
        gh.parse(filename, format="turtle")
mergttlfile(graph)
graph.serialize(destination='kg/kgraph215.ttl', format='turtle')
len(graph)

215


22431

     merge graphs     

In [149]:
len(graph)

22431

In [74]:
import pprint
for stmt in g:
    pprint.pprint(stmt)

(rdflib.term.URIRef(u'http://dbpedia.org/resource/Emetics'),
 rdflib.term.URIRef(u'http://dbpedia.org/ontology/wikiPageRedirects'),
 rdflib.term.URIRef(u'http://dbpedia.org/resource/Vomiting'))
(rdflib.term.URIRef(u'http://dbpedia.org/resource/Vomiting'),
 rdflib.term.URIRef(u'http://dbpedia.org/ontology/abstract'),
 rdflib.term.Literal(u'Erbrechen ist die schwallartige Entleerung des Magen- oder Speiser\xf6hreninhaltes (Chymus) entgegen der nat\xfcrlichen Richtung durch die Speiser\xf6hre und den Mund. Beim Erbrechen ziehen sich Magen und Zwerchfell und auch die Bauchmuskulatur zusammen, der Magenmund \xf6ffnet sich, so dass Mageninhalt, in schweren F\xe4llen auch Darminhalt, \xfcber die Speiser\xf6hre in den Mund und dann als Erbrochenes (Vomitat) weiter nach au\xdfen gelangt. Es ist meist mit einem brennenden Gef\xfchl in der Speiser\xf6hre (Sodbrennen) verbunden, das durch die Magens\xe4ure verursacht wird. Die medizinischen Fachbegriffe f\xfcr das Erbrechen sind die Emesis (Griech

See reference in RDFCreationTestingPage